# Nuevo esquema DB en Postgresql

EL siguiente Notebook será para cargar y preparar un esquema en postgresql con datos de productos relevantes para otros análisis

## En MySQL db8100

In [3]:
import MySQLdb
from datetime import datetime, timedelta
import pandas as pd

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()

In [4]:
cur.execute("""
    SELECT 
        NroID,
        Codigo,
        Nombre
    FROM dbp8100.formulas
""")

1187

In [5]:
# Obtener los resultados
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

In [6]:
df.head()

,NroID,Codigo,Nombre
0,35,None,FIBROMIXER 300 19% A.A. -1
1,36,3-K,VACA LECHERA 16% anulada no hacer
2,39,07-009,DEST. PRECOZ MEL. 0%A.A CASC DE SOJA
3,40,08-020,INICIADOR MELAZA (21 %A.A.)
4,41,09-H,C.M. URUGUAY REFORZADO 10%


In [7]:
df['melaza'] = 0
df['presentacion'] = ''
df.head()

,NroID,Codigo,Nombre,melaza,presentacion
0,35,None,FIBROMIXER 300 19% A.A. -1,0,
1,36,3-K,VACA LECHERA 16% anulada no hacer,0,
2,39,07-009,DEST. PRECOZ MEL. 0%A.A CASC DE SOJA,0,
3,40,08-020,INICIADOR MELAZA (21 %A.A.),0,
4,41,09-H,C.M. URUGUAY REFORZADO 10%,0,


## En Postgresql

In [8]:
import psycopg2

# Conexión a la base de datos PostgreSQL
conn_pg = psycopg2.connect(
    host="192.168.0.12",      # Cambia por la IP o hostname de tu servidor
    database="db_gral",  # Cambia por el nombre de tu base de datos
    user="apsa",        # Cambia por tu usuario
    password="Apsa2812"  # Cambia por tu contraseña
)

cur_pg = conn_pg.cursor()

In [9]:
# Crear la tabla en PostgreSQL (si no existe)
create_table_query = """
CREATE TABLE IF NOT EXISTS produccion.formulas (
    NroID INTEGER PRIMARY KEY,
    Codigo VARCHAR(50),
    Nombre TEXT,
    melaza INTEGER,
    presentacion TEXT
);
"""
cur_pg = conn_pg.cursor()
cur_pg.execute(create_table_query)
conn_pg.commit()

In [11]:
# Insertar los datos del DataFrame en la tabla
from io import StringIO

# Convertir el DataFrame a CSV en memoria
output = StringIO()
df.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
output.seek(0)

# Usar COPY para cargar los datos
cur_pg.copy_from(output, 'produccion.formulas', sep='\t', null='\\N', columns=df.columns)
conn_pg.commit()
cur_pg.close()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
